Helper functions copied from class hw.  Make sure you copy the chrome driver into this directory!

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib

#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver',options=options)


#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup

#helper function to write lists to files
def write_lst(lst,file_):
    with open(file_,'w') as f:
        for l in lst:
            f.write(l)
            f.write('\n')

Function to scrape links from an indeed search result

In [32]:
def scrape_search_result_page(dir_url,page_result,browser):
    print ('-'*20,'Scraping indeed search result page '+ str(page_result)+'','-'*20)
    indeed_links = []
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,browser) 
    for link_holder in soup.find_all('div',class_='title'): #get list of all <div> of class 'photo nocaption'
        rel_link = link_holder.find('a')['href'] #get url
        #url returned is relative, so we need to add base url
        if rel_link != '':
            indeed_links.append('https://www.indeed.com' + rel_link) 
    print ('-'*20,'Found {} indeed search urls'.format(len(indeed_links)),'-'*20)
    return indeed_links


Build query URL string and call the scraper function and loop over n number of pages

In [33]:
start = 0 #pagnigation variable, page 1 = 0, page 2 = 10, page 3 = 30, etc
q = 'project+manager' #job query string
l = 'New+York+State' #location of job
numPage = 5 #num pages to scrap links from
allLinks = [] # list to capture

for page_result in range(numPage):
    search_result_url = 'https://www.indeed.com/jobs?q='+ q +'&l='+ l +'&start='+str(start) #build query string
    jobSearchResult = scrape_search_result_page(search_result_url,page_result, browser) # call scraper function
    allLinks.extend(jobSearchResult) #add to link
    start =+10 #increment the variable used to denote the next page

-------------------- Scraping indeed search result page0 --------------------
-------------------- Found 17 indeed search urls --------------------
-------------------- Scraping indeed search result page1 --------------------
-------------------- Found 19 indeed search urls --------------------
-------------------- Scraping indeed search result page2 --------------------
-------------------- Found 19 indeed search urls --------------------
-------------------- Scraping indeed search result page3 --------------------
-------------------- Found 18 indeed search urls --------------------
-------------------- Scraping indeed search result page4 --------------------
-------------------- Found 18 indeed search urls --------------------


Print links and write to file for debug

In [29]:
print(allLinks)

job_urls_file = 'jobSearchResult' +q+'.txt'
write_lst(allLinks,job_urls_file)

['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Ac7vatbEYxaq9F1o01CGWcCKwcjf2gc0bJMWa9RG4RtWXD6yTkji-sv6V8BXpgKSNALYXYDArQms1p8_VFBELSIBRop7UKnSlGXL71tVjz8G6M9JV6tTso2QTjZQnjFMf0szi-mByvlXZCdCLVV6YOygWeYs4kxhgN2O0Kq1EuhV01ss7JehlwiD63HzoQ2ba59Iw3YWmI1dFIVETyjzZD0f2mSlSFe-5hRIqopjGkR10_jdrAgNz_JHoncbQyT7qcuWqA8ZKWohM3FMf_H1j3j0En54nY2EttUhXfgWNqjQjSYQMX4vNbsMhUl2K9nqDwUBOJZYYLrXOIcJSnmAU9-GaaPx2EwvevAUaG1j4d1PH4TG9csjqQAA1oD0QTzfoKE5FS7Seg58Wff-jc4BitNjFRv6Tbe7z891YMRgv-R5vONdpxHMgn1K3my2oVz0Q=&p=0&fvj=1&vjs=3', 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BpFtLZpysUYhlychQih9t3qgOjKabh_2uaPDPmk-KXh98kUE_Mbwcuk-KPuY_udyP6g3jryhDcgIvKpYrEawjJWBZGafFckUdKzCRSxcpMLLd2L8cVrYYPB54L_oTy41jFELTpsJQl_l93PfzWXJAziwDAMsBePkxzRHt6Cyjd6eNvHXjf6B9mfwvuoBgr7Edd84wS2Mz2xMVGGZ-ypnpjowQKW_lyXeACjrvkYgZenloK-4krnFv8K838sqKQzmqZtRfwk8sv8s9wN8hPTy3lwnIgWhxb3s0QCP7NLzhY6Pt2WbJNqf5y-Jhewg92o9gcRFc1mUhtsNVi2BchTqejAssCv2njy4O5Iz5q9C-D8Xc80q_IK2k4QAoVJ8YyMB-Ty6UZCY0ffuqrH6xE550UB98hl2ANmI0LcpLDWy

Scrape Page Data for each Link

In [58]:
homepage_found = False
page_data = ''
page_data_list = []
for indeed_url in allLinks:
    try:
        page_soup = remove_script(get_js_soup(indeed_url,browser)) 
    except:
        print ('Could not access {}'.format(indeed_url))
     
    page_data = process_bio(page_soup.get_text(separator=' '))  #helper function from bio hw to clean up text
    
    #remove header
    page_data = page_data[189:] #the 189 slice removes the header of the indeed pages
    
    #remove footer
    footer_position = page_data.find('save job') #find the position of 'save job' which starts the footer
    trimStringBy = footer_position - len(page_data) #returns a negative number to trim the string by
    page_data = page_data[:trimStringBy] #drop footer
    
    page_data_list.append(page_data) 
    

Print page data and write to file for debug

**Footer still has some text at the end which isn't properly cleaned

In [68]:
print(page_data_list)

page_data_file = 'pageText' +q+'.txt'
write_lst(page_data_list,page_data_file)

["Project Manager - Connectivity Consult Energy USA - New York, NY $75,000 - $85,000 a year -Full-time, Contract Commercial IoT Project Manager | NEW YORK | 75-85K DOE Job Description In this role you will work closely with the Global Operations Director, responsible for assisting, and eventually leading, pilot installs, understanding data reads, full client management and the tracking of project installs end to end. It is key that you can see the big picture and anticipate issues so projects are to delivered to time, quality and client scope. Essential skills/ experience Project management experience (c.2-3 years+) IT (Software and hardware) implementation experience Strong understanding of the operations lifecycle and role that IT plays in data connection Experience interacting with multiple operations departments Brilliant troubleshooting skills Good communication skills presentable to clients Decent data analysis skills Great attitude, keen to learn Happy to travel to client sites 